In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame(
    {
        "A": [1, 2, 3, np.nan, 5, np.nan, 1],
        "B": [6, 7, np.nan, 9, 10, 6, 0],
        "C": ["a ba", "a", "c", "c", "b", "a", "a"],
        "D": [1, 3, 4, 4, 2, 2, 2],
    }
)
df

,A,B,C,D
0,1.0,6.0,a ba,1
1,2.0,7.0,a,3
2,3.0,NaN,c,4
3,NaN,9.0,c,4
4,5.0,10.0,b,2
5,NaN,6.0,a,2
6,1.0,0.0,a,2


# Filter, String

In [ ]:
def article_views(views: pd.DataFrame) -> pd.DataFrame:
    return (
        views[views["author_id"] == views["viewer_id"]][["author_id"]]
        .drop_duplicates()
        .rename(columns={"author_id": "id"})
        .sort_values(by="id")
    )


def article_views_v2(views: pd.DataFrame) -> pd.DataFrame:
    authors = sorted(
        views[views["author_id"] == views["viewer_id"]]["author_id"].unique()
    )
    return pd.DataFrame({"id": authors})

In [9]:
df["C"].str.split().apply(lambda l: any(val.startswith("b") for val in l))

0     True
1    False
2    False
3    False
4     True
5    False
6    False
Name: C, dtype: bool

In [ ]:
def find_patients(patients: pd.DataFrame) -> pd.DataFrame:
    return patients[
        patients["conditions"]
        .str.split()
        .apply(lambda l: any(val.startswith("DIAB1") for val in l))
    ]


def find_patients_v2(patients: pd.DataFrame) -> pd.DataFrame:
    return patients[
        patients["conditions"].str.startswith("DIAB1")
        | patients["conditions"].str.contains(" DIAB1")
    ]


def find_patients_v3(patients: pd.DataFrame) -> pd.DataFrame:
    return patients[patients["conditions"].str.contains(r"\bDIAB1")]

In [ ]:
def valid_emails(users: pd.DataFrame) -> pd.DataFrame:
    return users[
        users["mail"].str.contains(r"^[a-zA-Z][a-zA-Z0-9_.-]*@leetcode\.com$")
    ]

# Agg

In [ ]:
def daily_leads_and_partners(daily_sales: pd.DataFrame) -> pd.DataFrame:
    return daily_sales.groupby(["date_id", "make_name"], as_index=False).agg(
        unique_leads=("lead_id", "nunique"),
        unique_partners=("partner_id", "nunique"),
    )

In [16]:
list_df = df.groupby("C", as_index=False)["D"].apply(list)
list_df

,C,A
0,a,"[2.0, nan, 1.0]"
1,a ba,[1.0]
2,b,[5.0]
3,c,"[3.0, nan]"


In [26]:
df.groupby("C", as_index=False).agg({"D": lambda s: sorted(s.values)})

,C,D
0,a,"[2, 2, 3]"
1,a ba,[1]
2,b,[2]
3,c,"[4, 4]"


In [27]:
df.groupby("C", as_index=False).agg(ngok=("D", lambda s: sorted(s.values)))

,C,ngok
0,a,"[2, 2, 3]"
1,a ba,[1]
2,b,[2]
3,c,"[4, 4]"


In [17]:
list_df["A"].str.len()

0    3
1    1
2    1
3    2
Name: A, dtype: int64

In [ ]:
def categorize_products(activities: pd.DataFrame) -> pd.DataFrame:
    grouped_df = activities.groupby("sell_date", as_index=False).agg(
        products=("product", lambda s: ",".join(sorted(s.unique())))
    )
    grouped_df["num_sold"] = grouped_df["products"].str.len()
    return (
        grouped_df[["sell_date", "num_sold", "products"]]
        .sort_values(by="sell_date")
        .reset_index(drop=True)
    )


def categorize_products_v2(activities: pd.DataFrame) -> pd.DataFrame:
    grouped_df = activities.groupby("sell_date").agg(
        num_sold=("product", "unique"),
        products=("product", lambda s: ",".join(sorted(s.unique()))),
    )
    return grouped_df.reset_index().sort_values(by="sell_date")


def categorize_products_v3(activities: pd.DataFrame) -> pd.DataFrame:
    grouped_df = (
        activities.groupby("sell_date")["product"]
        .agg(
            [
                "nunique",
                lambda x: ",".join(sorted(x.unique())),
            ]
        )
        .reset_index()
    )
    grouped_df.columns = ["sell_date", "num_sold", "products"]
    return grouped_df.sort_values(by="sell_date")

In [30]:
df.groupby("C")["A"].count().max()

2

In [ ]:
df.groupby("C")["A"].count().idxmax()

In [40]:
df["C"].mode()

0    a
Name: C, dtype: object

In [42]:
df["C"].mode().to_frame(name="count")

,count
0,a


In [ ]:
def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    if len(orders) == 0:
        return pd.DataFrame({"customer_number": []})
    customer_number = (
        orders.groupby("customer_number")["order_number"].count().idxmax()
    )
    return pd.DataFrame({"customer_number": [customer_number]})


def largest_orders_v2(orders: pd.DataFrame) -> pd.DataFrame:
    return orders["customer_number"].mode().to_frame()

# Rename, Fill

In [3]:
df_2 = pd.DataFrame(
    {
        "C": ["a", "b", "c"],
        "E": ["A", "B", "C"],
    }
)

In [4]:
df.rename(columns={"A": "AA"})

,AA,B,C,D
0,1.0,6.0,a,1
1,2.0,7.0,a,3
2,3.0,NaN,c,4
3,NaN,9.0,c,4
4,5.0,10.0,b,2
5,NaN,6.0,a,2
6,1.0,0.0,a,2


In [10]:
df["D"].astype(np.int32)
df["D"].astype(np.float64)

0    1.0
1    3.0
2    4.0
3    4.0
4    2.0
5    2.0
6    2.0
Name: D, dtype: float64

In [7]:
df["A filled"] = df["A"].fillna(value=0)

In [17]:
a_mean = int(df["A"].mean())
a_mean

1

In [22]:
df.fillna(value={"A": df["A"].mean()})

,A,B,C,D
0,1.0,6.0,a,1
1,2.0,7.0,a,3
2,3.0,NaN,c,4
3,2.4,9.0,c,4
4,5.0,10.0,b,2
5,2.4,6.0,a,2
6,1.0,0.0,a,2


In [24]:
df.fillna(value=df[["A", "B"]].mean())

,A,B,C,D
0,1.0,6.000000,a,1
1,2.0,7.000000,a,3
2,3.0,6.333333,c,4
3,2.4,9.000000,c,4
4,5.0,10.000000,b,2
5,2.4,6.000000,a,2
6,1.0,0.000000,a,2


In [8]:
df.fillna(value={"A": 0})

,A,B,C,D
0,1.0,6.0,a,1
1,2.0,7.0,a,3
2,3.0,NaN,c,4
3,0.0,9.0,c,4
4,5.0,10.0,b,2
5,0.0,6.0,a,2
6,1.0,0.0,a,2


In [ ]:
def renameColumns(students: pd.DataFrame) -> pd.DataFrame:
    return students.rename(
        columns={
            "id": "student_id",
            "first": "first_name",
            "last": "last_name",
            "age": "age_in_years",
        }
    )

In [ ]:
def changeDatatype(students: pd.DataFrame) -> pd.DataFrame:
    students["grade"] = students["grade"].astype(int)
    return students


def changeDatatype_v2(students: pd.DataFrame) -> pd.DataFrame:
    return students.astype({"grade": "int"})


def changeDatatype_v3(students: pd.DataFrame) -> pd.DataFrame:
    return students.assign(grade=students["grade"].astype(int))

In [ ]:
def fillMissingValues(products: pd.DataFrame) -> pd.DataFrame:
    products["quantity"].fillna(value=0, inplace=True)
    return products


def fillMissingValues_v2(products: pd.DataFrame) -> pd.DataFrame:
    products["quantity"] = products["quantity"].fillna(value=0)
    return products

In [51]:
"a" in df["C"].values

True

In [46]:
df[df["C"] == "e"]["C"].values

array([], dtype=object)

In [ ]:
def sales_person(
    sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame
) -> pd.DataFrame:
    if "RED" not in company["name"].values:
        return sales_person[["name"]]
    red_id = company[company["name"] == "RED"]["com_id"].values[0]
    red_sales = orders[orders["com_id"] == red_id]["sales_id"]
    return sales_person[~sales_person["sales_id"].isin(red_sales)][["name"]]

# Reshape

In [28]:
df1 = pd.DataFrame(
    {
        "C": ["a", "b", "c"],
        "E": ["A", "B", "C"],
    }
)
df2 = pd.DataFrame(
    {
        "C": ["a1", "b1"],
        "E": ["A1", "B1"],
    }
)
# pd.concat([df1, df2])
pd.concat([df1, df2], axis="index", ignore_index=True)

,C,E
0,a,A
1,b,B
2,c,C
3,a1,A1
4,b1,B1


In [29]:
df1 = pd.DataFrame(
    {
        "C": ["a", "b", "c"],
        "E": ["A", "B", "C"],
    }
)
df2 = pd.DataFrame(
    {
        "A": [4, 5, 6],
        "B": [1, 2, 3],
    }
)
pd.concat([df1, df2], axis="columns")

,C,E,A,B
0,a,A,4,1
1,b,B,5,2
2,c,C,6,3


In [35]:
temp_df = df.groupby(["D", "C"], as_index=False)["A"].sum()
temp_df.pivot(index="D", columns="C", values="A")

C,a,b,c
D,,,
1,1.0,NaN,NaN
2,1.0,5.0,NaN
3,2.0,NaN,NaN
4,NaN,NaN,3.0


In [40]:
df.pivot_table(index="D", columns="C", values="A", aggfunc="max").reset_index()

C,D,a,b,c
0,1,1.0,NaN,NaN
1,2,1.0,5.0,NaN
2,3,2.0,NaN,NaN
3,4,NaN,NaN,3.0


In [43]:
pivot_df = df.pivot_table(
    index="D", columns="C", values="A", aggfunc="max"
).reset_index()
pivot_df.melt(
    id_vars=["D"], value_vars=["a", "b", "c"], var_name="CC", value_name="AA"
)

,D,CC,AA
0,1,a,1.0
1,2,a,1.0
2,3,a,2.0
3,4,a,NaN
4,1,b,NaN
5,2,b,5.0
6,3,b,NaN
7,4,b,NaN
8,1,c,NaN
9,2,c,NaN


In [46]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [ ]:
def pivotTable(df: pd.DataFrame) -> pd.DataFrame:
    return df.pivot(index="month", columns="city", values="temperature")


def meltTable(report: pd.DataFrame) -> pd.DataFrame:
    return report.melt(
        id_vars=["product"],
        value_vars=["quarter_1", "quarter_2", "quarter_3", "quarter_4"],
        var_name="quarter",
        value_name="sales",
    )


def meltTable_v2(report: pd.DataFrame) -> pd.DataFrame:
    return report.melt(
        id_vars=["product"],
        value_vars=[col for col in report.columns if col != "product"],
        var_name="quarter",
        value_name="sales",
    )

In [ ]:
def findHeavyAnimals(animals: pd.DataFrame) -> pd.DataFrame:
    return animals.loc[animals["weight"] > 100].sort_values(
        by="weight", ascending=False
    )[["name"]]